In [25]:
# create file slowly
import os
import sys

import numpy as np

path = os.path.split(os.getcwd())[0]
if path not in sys.path:
    sys.path.append(path)

from h5loader import H5File

In [53]:
ls

Demo file creation 1.ipynb  test.bin                 test.npy
Demo file creation.ipynb    Testing for gui 1.ipynb
Memap creation 1.ipynb      test.meta


In [27]:
test_filename = '/media/jonathan/DATA/seizure_data/gabrielle/All_DATA/EEG DATA CRISPRa Kcna1 2018/4_CRISP Oct-Nov 2018/CRISPRa_h5s TREATMENT/M1540639362_2018-10-27-12-22-42_tids_[102, 158, 162, 163, 165, 166, 170].h5'

In [54]:
import shutil

In [58]:
shutil.copy(test_filename,'M1540639362_2018-10-27-12-22-42_tids_[102, 158, 162, 163, 165, 166, 170].h5' )

'M1540639362_2018-10-27-12-22-42_tids_[102, 158, 162, 163, 165, 166, 170].h5'

In [59]:
h5 = H5File(test_filename)

In [60]:
fs_dict = eval(h5.attributes['fs_dict'])
fs = fs_dict[int(h5.attributes['t_ids'][0])]
channels = []
for tid in h5.attributes['t_ids']:
    assert fs == int(fs_dict[tid])
    channels.append(h5[tid]['data'])

arr = np.vstack(channels).T

In [64]:
arr.shape[0]/256, fs, len(channels)

(3600.0, 256.0, 7)

In [71]:
arr_i4 = arr.astype('>i4')
arr_i4.tofile('M1540639362_2018-10-27-12-22-42.leite')

In [72]:
from collections import OrderedDict
import json
metadata = OrderedDict(binaryfilename = 'M1540639362_2018-10-27-12-22-42.leite',
                       fs= int(fs),
                       no_channels= len(channels),
                       no_time_points= 3600*int(fs),
                       data_format='>i4',
                       volts_per_bit= 2.5/6/2**31, 
                       transmitter_id= '001',
                       start_timestamp_unix= 1564066727,
                       channel_labels=[[102, 158, 162, 163, 165, 166, 170]],
                       experiment_metadata_str='kainate_acid_hippo')

def save_metadata_as_json(obj,fp):
    if not fp.endswith('.meta'):
        if fp.endswith('.bin'):
            fp = fp[:-4] 
        fp = fp + '.meta'
        
    print(fp)
    with open(fp, 'w') as json_file:
        json.dump(obj,json_file, indent=2, sort_keys=True)

In [70]:
save_metadata_as_json(metadata,'M1540639362_2018-10-27-12-22-42.meta')

M1540639362_2018-10-27-12-22-42.meta


# Use the h5 file to make a live file:

In [50]:
update_speed = 0.1
with open('test_live.npy', 'wb') as f:
    for i in range(int(3600//update_speed)):
        fs = int(256*update_speed)
        buffer = arr[i*fs:(i+1)*fs, :arr.shape[1]]
        time.sleep(update_speed)
        buffer = buffer.astype('>i4')
        buffer.tofile(f)

KeyboardInterrupt: 

In [52]:
pwd

'/home/jonathan/git_repos/pyecog2/Notebooks'

In [5]:
#%%timeit
with open('test.npy', 'wb') as f:
    for i in range(10):
        new_data = np.random.randn(512)*1000
        new_data = new_data.astype('>i4')
        new_data.tofile(f)

In [6]:
import time

In [7]:
%%time
n_channels = 4
with open('test.dat', 'wb') as f:
    time.sleep(0.5)
    for i in range(3600):
        time.sleep(0.1)
        new_data = np.random.randn(512*n_channels)*1000
        new_data = new_data.astype('>i4')
        new_data.tofile(f)

KeyboardInterrupt: 

In [61]:
%%timeit
with open('test.npy', 'wb') as f:
    for i in range(1000):
        new_data = np.random.randn(512)*1000
        new_data = new_data.astype('>i4')
        new_data.tofile(f)

102 ms ± 1.38 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [1]:
#%%timeit
with open('test.npy', 'wb') as f:
    for i in range(3600):
        new_data = np.random.randn(4*512)*1000
        new_data = new_data.astype('>i4')
        new_data.tofile(f)

NameError: name 'np' is not defined

In [53]:
np.memmap('test.npy',dtype='>i4').shape

(20480,)

In [54]:
np.memmap('test.npy',dtype='int32').shape

(20480,)

In [55]:
new_data

array([-1602,   708,   518, ...,   674,   254,  -946], dtype=int32)

In [56]:
10*4*512

20480